# Imports

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

# Dask
import dask.dataframe as dd

# pyarrow and feather
import pyarrow.feather as feather
import pyarrow.dataset as ds

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

C:\Users\Jessie\miniconda3\envs\525_2022\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


# 1. Teamwork Contract
The teamwork contract for our team, group 7, can be found [here](https://docs.google.com/document/d/1u4e5Z5C-uwTTSvCEyOYy-I30Fb8OEPYM6frM0NBEVVc/edit).

# 2. Create repository and project structure
The repository URL: https://github.com/UBC-MDS/DSCI525-Group7

# 3. Downloading the data

Using Python **requests** Library

We are using article id #14096681, which contains the data of **Daily rainfall over NSW, Australia.**

In [3]:
# Setup
article_id = 14096681  
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "rainfall/"

Review the files within the article:

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

# 3.1 Unzipping the data

In [5]:
%%time

files_to_dl = ["data.zip"]  
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: total: 8.48 s
Wall time: 1min 35s


In [7]:
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

In [8]:
%ls -ltr rainfall/

Invalid switch - "".


# Comparison of Performance on Different Machines

The summary of all team members' time taken to unzip the data is recorded below. Each team member's Operating System, RAM, Processor and SSD are also recorded to check if they have any effect on the time taken.

| Team Member | Operating System     | RAM       | Processor                                                      | Is SSD   | Time Taken |
| ----------- | -----------          |-----------| ---------- ---------------------------------------------------|----------|---------  -|
| Jessie      | Windows 10 Education |  16GB      |   Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz   1.99 GHz        |   Yes    |  CPU times: total: 8.48 s <br> Wall time: 1min 35s          |
| Adrianne    |                      |           |                                                               |           |           |
| Rada        |                      |           |                                                               |           |           |
| Moid        |                      |           |                                                                |           |           |

>**Discussion of results above**

# 4. Combining data CSVs

- Combine data CSVs into a single CSV using pandas.

- When combining the CSV files, add an extra column called "model" that identifies the model. Tip 1: you can get this column populated from the file name, eg: for file name "SAM0-UNICON_daily_rainfall_NSW.csv", the model name is SAM0-UNICON Tip 2: Remember how we added year when we combined airline CSVs. Tip 3: You can use regex generator.

_Note: There is a file called observed_daily_rainfall_SYD.csv in the data folder that you downloaded. Make sure you exclude this file (programmatically or just take out that file from folder) before you combine CSVs. We will use this file in our next milestone._

- Compare run times on different machines within your team and summarize your observations.
Warning: Some of you might not be able to do it on your laptop. It's fine if you're unable to do it. Just make sure you discuss the reasons why you might not have been able to run this on your laptop.

Let's first view the data and the columns:

In [9]:
%%time

df_1 = pd.read_csv(output_directory+"/MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv")
df_2 = pd.read_csv(output_directory+"/CMCC-CM2-SR5_daily_rainfall_NSW.csv")
df_3 = pd.read_csv(output_directory+"/SAM0-UNICON_daily_rainfall_NSW.csv")

CPU times: total: 7.16 s
Wall time: 7.22 s


Even loading three of the individual files is taking a little time.

In [10]:
df_1.head(2)

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13


In [11]:
df_2.head(2)

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-35.811518,-34.86911,140.625,141.875,0.000424
1,1889-01-02 12:00:00,-35.811518,-34.86911,140.625,141.875,0.006158


In [12]:
df_3.head(2)

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-35.811518,-34.86911,140.625,141.875,3.045650e-13
1,1889-01-02 12:00:00,-35.811518,-34.86911,140.625,141.875,3.572392e-04


In [13]:
df_3.tail(2)

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
3541151,2014-12-30 12:00:00,-30.157068,-29.21466,153.125,154.375,8.541592
3541152,2014-12-31 12:00:00,-30.157068,-29.21466,153.125,154.375,68.117489


In [14]:
%%time

files = glob.glob('rainfall/*NSW.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall(r'/([^_]*)', file)[0])
                for file in files)
              )
df.to_csv("rainfall/combined_data.csv")

IndexError: list index out of range

In [20]:
# For Windows user 
##  Windows users will run into an index error when running the code above to combine the CSVs. This can be solved by adding a ./ to the filename as below.

In [18]:
%%time
%memit
files = glob.glob('./rainfall/*NSW.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=file.strip('./rainfall\\').split('_')[0])
                for file in files)
              )
df.to_csv("rainfall/combined_data.csv")

peak memory: 800.19 MiB, increment: 0.00 MiB
CPU times: total: 8min 45s
Wall time: 8min 55s


Wow, this felt like an eternity!

Let's take a look at the combined file, see if head and tail are as we expect them to be:

In [21]:
df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


In [22]:
df.tail()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
2014-12-27 12:00:00,-30.157068,-29.21466,153.125,154.375,0.554375,TaiESM1
2014-12-28 12:00:00,-30.157068,-29.21466,153.125,154.375,7.028577,TaiESM1
2014-12-29 12:00:00,-30.157068,-29.21466,153.125,154.375,0.234757,TaiESM1
2014-12-30 12:00:00,-30.157068,-29.21466,153.125,154.375,2.097459,TaiESM1
2014-12-31 12:00:00,-30.157068,-29.21466,153.125,154.375,0.548421,TaiESM1


## Comparison of Performance on Different Machines

The summary of all team members' time taken to combine the CSV's files is recorded below. Each team member's Operating System, RAM, Processor and SSD are also recorded to check if they have any effect on the time taken.

| Team Member | Operating System     | RAM       | Processor                                                      | Is SSD   | Time Taken |
| ----------- | -----------          |-----------| ---------- ---------------------------------------------------|----------|---------  -|
| Jessie      | Windows 10 Education |  16GB      |   Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz   1.99 GHz        |   Yes    |  peak memory: 800.19 MiB  increment: 0.00 MiB <br> CPU times: total: 8min 33s <br> Wall time: 8min 40s          |
| Adrianne    |                      |           |                                                               |           |           |
| Rada        |                      |           |                                                               |           |           |
| Moid        |                      |           |                                                                |           |           |

> **Discussion of results above** 

# 5. Load the combined CSV to memory and perform a simple EDA

1. Investigate at least two of the following approaches to reduce memory usage while performing the EDA (e.g., value_counts).

- Changing dtype of your data
- Load just columns what we want
- Loading in chunks
- Dask

2. Compare run times on different machines within your team and summarize your observations.

**The EDA will be to use value_counts() to count the number of data points that came from each .csv file, as recorded in the model column of combined_data.csv.**

### 5.1 Load the Entire Dataframe to Memory Using Pandas (Baseline for Comparison)

In [27]:
%%time
%%memit

df_pandas = pd.read_csv("rainfall/combined_data.csv")
print(df_pandas["model"].value_counts())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
peak memory: 9060.10 MiB, increment: 4510.68 MiB
CPU times: total: 1min 28s
Wall time: 1min 31s


**Observations**
>Our baseline approach is to use Pandas to load the entire data to memory. The above code loads the combined_data.csv to memory and performs a simple EDA to calculate counts of values in the "model" column. We see that the peak memory is 9060 MiB and the CPU and wall time is 1min 31s. We will explore some other approaches to see if we can reduce the time and memory usage.

### 5.2 Changing dtypes of data:

- We will attempt to change time column from datetime to date
- We will attempt to read the numerical columns using float32 format

Memory comparison for format changes adapted from Lecture notes:

In [ ]:
df.index = pd.to_datetime(df.index).dt.date

In [23]:
print(f"Memory usage with float64: {df[['lat_min','lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']].memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df[['lat_min','lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']].astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 2998.46 MB
Memory usage with float32: 1749.10 MB


In [ ]:
df_float32 = df.copy()
df_float32[['lat_min','lat_max','lon_min', 'lon_max', 'rain (mm/day)']].astype('float32')

#saving the dataframe of float32 to file
df_float32.to_csv("rainfall/combined_data_float32.csv")

In [ ]:
%%time
%%memit

#loading the float32 dataframe to memory and perform a simple EDA for value counts of model column
df_float32 = pd.read_csv("rainfall/combined_data_float32.csv")
print(df_float32["model"].value_counts())

**Observations:**
> When we changed the data type from float64 to float32 the memory usage reduced by nearly half. This is because float32 is stored as a 32-bit number, while float64 is stored as 64-bit number, which is twice as much memory as float32. With the EDA, we see that after converting dtypes to float 32, the peak memory usage decreased to and both CPU and wall time in/decreased.Changing the dtype is effective in reducing the time and memory required to load data, and should be used when we have a large amount of data that does not require very high precision.





### 5.3 Dask:

- We will attempt to read dataframe using dask

In [24]:

%%time
%%memit
# Dask
df_dask = dd.read_csv('rainfall/combined_data.csv')
print(df_dask["model"].value_counts().compute())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
peak memory: 4749.06 MiB, increment: 1255.98 MiB
CPU times: total: 1min 8s
Wall time: 23.5 s


**Observations:**
> Using a Dask dataframe is much faster and lighter on memory. Compared to loading the csv to pandas data frame, when we load the csv file to dask, the peak memory, increment memory, and wall time all reduced significantly when calling the value_counts() function. This is likely because dask partitioned the dataframe based on row index and did the calculation in parallel to improve the efficiency. Thus, for large-scale data calculation, we could use dask instead of pandas to improve the code efficiency with minimal syntax change.

### 5.4 Loading in Chunks:

- We will attempt to read dataframe in chunks

#### Chunksize = 10 million:

In [26]:
%%time
%%memit
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("rainfall/combined_data.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int32
peak memory: 6795.32 MiB, increment: 2462.11 MiB
CPU times: total: 1min 14s
Wall time: 1min 15s


#### Chunksize = 1 million:

In [33]:
%%time
%%memit
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("rainfall/combined_data.csv", chunksize=1_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int32
peak memory: 3741.75 MiB, increment: 254.97 MiB
CPU times: total: 1min 22s
Wall time: 1min 23s


**Observations:**
> When loading the data in chunks, the peak memory is significantly lower than that without using chunking method. We can see that loading in 10 million per chunk requires nearly 6800 MiB in peak memory usage, which is less than using Pandas to load all at once. Loading in 1 million per chunk requires only 4,235 MiB in peak memory usage. This is significantly more efficient than using Pandas. However, we also notice that the CPU and wall time remains roughly the same in all these approaches.




### 5.5 Selecting columns:

Since we only want the model for EDA, we will import just the model column. This is faster and uses less memory than loading the whole dataframe.


In [ ]:
%%time
%%memit
df = pd.read_csv("rainfall/combined_data.csv", 
                 usecols = ["model"])

In [ ]:
%%time
%%memit
df["model"].value_counts()

**Observations:**
>Running value_counts takes the same time as it did using the entire data set, probably because it has to iterate through the same number of rows. However, this should still be done whenever possible because it reduces memory required and speeds up loading data

# 6. Perform a simple EDA in R

To perform EDA in R, we first need to transfer the dataframe from Python to R.
In this section, we will pass data from python to R in various ways and asses each method.